In [1]:
# Libraries for analysis
import numpy as np
import cv2 as cv
import csv

# Libraries for visuals
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(font_scale = 1.2)

# Allows charts to appear in the notebook
%matplotlib inline

In [20]:
def DomOriMag(u_crop, v_crop, ori_res):
    
    mag, ang = cv.cartToPolar(u_crop, v_crop, angleInDegrees = True)
    
    ang = np.floor(ang/ori_res) # Quantizing the angles to 360/res bins
    ang = np.reshape(ang, (1, -1))
    
    mag = np.around(mag) # Because we don't need float percision for magnitude, we round them here
    mag = np.reshape(mag, (1, -1))
    
    oriBins = np.uint8(360/ori_res)
    OriMagHist = np.zeros(oriBins)
    for i in range(oriBins):
        indexes = np.where(ang == i)
        OriMagHist[i] = np.sum(mag[indexes])
        
    domOriTemp = np.argmax(OriMagHist)
    domOriMagsIdx = np.where(ang == domOriTemp)
    domOriMags = mag[domOriMagsIdx]
    
    maxMag = np.max(domOriMags)
    magHist, bins = np.histogram(domOriMags, bins = np.arange(maxMag + 2))
    
    return domOriTemp * ori_res, magHist

In [21]:
def childFlow(frame1_gray, frame2_gray, bbox, direction):
    
    # In following lines, we will do two sets of things. First, due to our previous angle,
    # we will choose whether extend our checking bbox in the right side or left side. Second,
    # we check if our bbox is going outside or not and then, correcting its bounderies
    
    bbox_new = np.copy(bbox)
    # Check right side movement of bbox
    if direction == "right":
        bbox_new = np.array([bbox[0], bbox[1], bbox_l, 2*bbox_w])
        bbox_new = np.clip(bbox_new, [0, 0, 0, 0], [frame_l, frame_w, frame_l, frame_w])
    elif direction == "left":
        bbox_new = np.array([bbox[0], bbox[1]-bbox_w, bbox_l, 2*bbox_w])
        bbox_new = np.clip(bbox_new, [0, 0, 0, 0], [frame_l, frame_w, frame_l, frame_w])
    
    # Now, we will calculate optical flow and take our crop from previous step
    
    subtFrames = cv.absdiff(frame2_gray, frame1_gray)
    ret, thresh = cv.threshold(subtFrames, 25, 255, cv.THRESH_BINARY)
    
    flow = cv.calcOpticalFlowFarneback(frame1_gray, frame2_gray, None, 0.5, 5, 15, 3, 5, 1.2, 0)
    
    flow_u = flow[...,0] * thresh/255
    flow_v = flow[...,1] * thresh/255
            
    u_crop = flow_u[bbox_new[0]:bbox_new[0]+bbox_new[2], bbox_new[1]:bbox_new[1]+bbox_new[3]]
    v_crop = flow_v[bbox_new[0]:bbox_new[0]+bbox_new[2], bbox_new[1]:bbox_new[1]+bbox_new[3]]
    
    return u_crop, v_crop

In [22]:
def childFlow_1(frame1_gray, frame2_gray, bbox, direction):
    
    # In following lines, we will do two sets of things. First, due to our previous angle,
    # we will choose whether extend our checking bbox in the right side or left side. Second,
    # we check if our bbox is going outside or not and then, correcting its bounderies
    
    bbox_new = np.copy(bbox)
    # Check right side movement of bbox
    if direction == "right":
        bbox_new = np.array([bbox[0], bbox[1], bbox_l, 2*bbox_w])
        bbox_new = np.clip(bbox_new, [0, 0, 0, 0], [frame_l, frame_w, frame_l, frame_w])
    elif direction == "left":
        bbox_new = np.array([bbox[0], bbox[1]-bbox_w, bbox_l, 2*bbox_w])
        bbox_new = np.clip(bbox_new, [0, 0, 0, 0], [frame_l, frame_w, frame_l, frame_w])
    
    # Now, we will calculate optical flow and take our crop from previous step
    fr1_gray_crop = frame1_gray[bbox_new[0]:bbox_new[0]+bbox_new[2], bbox_new[1]:bbox_new[1]+bbox_new[3]]
    fr2_gray_crop = frame2_gray[bbox_new[0]:bbox_new[0]+bbox_new[2], bbox_new[1]:bbox_new[1]+bbox_new[3]]
    
    subtFrames = cv.absdiff(fr2_gray_crop, fr1_gray_crop)
    ret, thresh = cv.threshold(subtFrames, 25, 255, cv.THRESH_BINARY)
    
    flow = cv.calcOpticalFlowFarneback(fr1_gray_crop, fr2_gray_crop, None, 0.5, 5, 15, 3, 5, 1.2, 0)
    
    u_crop = flow[...,0] * thresh/255
    v_crop = flow[...,1] * thresh/255
    
    return u_crop, v_crop

In [23]:
def childFlow_2(frame1_gray, frame2_gray, bbox, direction):
    
    lk_params = dict( winSize  = (15,15),
                      maxLevel = 4,
                      criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 1, 0.03))
    
    bbox_new = np.copy(bbox)
    
    # Check right side movement of bbox
    if direction == "right":
        bbox_new = np.array([bbox[0], bbox[1], bbox_l, 2*bbox_w])
        bbox_new = np.clip(bbox_new, [0, 0, 0, 0], [frame_l, frame_w, frame_l, frame_w])
    elif direction == "left":
        bbox_new = np.array([bbox[0], bbox[1]-bbox_w, bbox_l, 2*bbox_w])
        bbox_new = np.clip(bbox_new, [0, 0, 0, 0], [frame_l, frame_w, frame_l, frame_w])
        
    # Now, we will calculate optical flow and take our crop from previous step
    
    subtFrames = cv.absdiff(frame2_gray, frame1_gray)
    ret, thresh = cv.threshold(subtFrames, 25, 255, cv.THRESH_BINARY)

    flow_u = np.zeros((np.size(frame1_gray, 0), np.size(frame1_gray, 1)))
    flow_v = np.zeros((np.size(frame1_gray, 0), np.size(frame1_gray, 1)))
    
    ptsRow, ptsCol = np.nonzero(thresh)
    pts0 = np.transpose(np.array([ptsRow, ptsCol], dtype=np.float32))
    
    pts1, st, err = cv.calcOpticalFlowPyrLK(frame1_gray, frame2_gray, pts0, None, **lk_params)
    e, f = np.nonzero(st)
    good_pts0 = pts0[e]
    good_pts1 = pts1[e]
    
    u_v = (good_pts1 - good_pts0)
    flow_u[np.int16(good_pts0[:, 0]), np.int16(good_pts0[:, 1])] = u_v[:, 0]
    flow_v[np.int16(good_pts0[:, 0]), np.int16(good_pts0[:, 1])] = u_v[:, 1]
    
    u_crop = flow_u[bbox_new[0]:bbox_new[0]+bbox_new[2], bbox_new[1]:bbox_new[1]+bbox_new[3]]
    v_crop = flow_v[bbox_new[0]:bbox_new[0]+bbox_new[2], bbox_new[1]:bbox_new[1]+bbox_new[3]]
    
    return u_crop, v_crop

In [24]:
def oriJumpReduct(val, prv_val, cntr):
    if np.abs(val - prv_val) > 90 and np.abs(val - prv_val) < 270:
        if cntr == 0:
            val = prv_val
            cntr += 1
        else:
            cntr = 0
    else:
        cntr = 0
    
    return val, cntr

In [25]:
def MagOriToMove(domMag, domOri):
    
    rowMove = domMag * np.sin(np.deg2rad(domOri))
    colMove = domMag * np.cos(np.deg2rad(domOri))
    rowMove = np.int32(rowMove)
    colMove = np.int32(colMove)
    
    return rowMove, colMove

In [26]:
def MoveComp(frame1_gray, frame2_gray, domMag):
    
    subtFrames = cv.subtract(frame1_gray, frame2_gray)
    ret, threshold = cv.threshold(subtFrames, 25, 255, cv.THRESH_BINARY)
    
    w, x, y, z = bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]
    w, x, y, z = np.clip(np.array([w, x, y, z]), [0, 0, 0, 0], [frame_l, frame_w, frame_l, frame_w])
    
    up, left, down, right = 0, 0, 0, 0
    if bbox[0] > 0:
        up = threshold[w, x:z]
    if bbox[1] > 0:
        left = threshold[w:y, x]
    if bbox[0] + bbox[2] < 1080:
        down = threshold[y, x:z]
    if bbox[1] + bbox[3] < 1920:
        right = threshold[w:y, z]
    
    if domMag < 40:
        domMag = 40
    
    rowComp, colComp = 0, 0
    
    if np.count_nonzero(up) > 0:
        rowComp -= domMag * 0.75
    if np.count_nonzero(left) > 0:
        colComp -= domMag * 0.75
    if np.count_nonzero(down) > 0:
        rowComp += domMag * 0.75
    if np.count_nonzero(right) > 0:
        colComp += domMag * 0.75
    
    rowComp = np.int32(rowComp)
    colComp = np.int32(colComp)
    
    return rowComp, colComp

In [27]:
def MoveComp_1(frame1_gray, frame2_gray, domMag):
    
    subtFrames = cv.subtract(frame1_gray, frame2_gray)
    ret, threshold = cv.threshold(subtFrames, 25, 255, cv.THRESH_BINARY)
    
    w, x, y, z = bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]
    w, x, y, z = np.clip(np.array([w, x, y, z]), [0, 0, 0, 0], [frame_l, frame_w, frame_l, frame_w])
    
    up, left, down, right = 0, 0, 0, 0
    if bbox[0] > 0:
        up = threshold[w, x:z]
    if bbox[1] > 0:
        left = threshold[w:y, x]
    if bbox[0] + bbox[2] < 1080:
        down = threshold[y, x:z]
    if bbox[1] + bbox[3] < 1920:
        right = threshold[w:y, z]
    
    if domMag < 40:
        domMag = 40
    
    rowComp, colComp = 0, 0
    
    if np.count_nonzero(up) > 0:
        rowComp -= domMag * 0.75
    if np.count_nonzero(left) > 0:
        colComp -= domMag * 1
    if np.count_nonzero(down) > 0:
        rowComp += domMag * 0.75
    if np.count_nonzero(right) > 0:
        colComp += domMag * 1
        
    if np.count_nonzero(right) > 0 and np.count_nonzero(left) > 0:
        if np.count_nonzero(right) > np.count_nonzero(left):
            colComp += domMag * 1
        elif np.count_nonzero(right) < np.count_nonzero(left):
            colComp -= domMag * 1
    
    if bbox[0] > frame_l - bbox_l:
        if np.count_nonzero(threshold[w+10, x:z]) < 10:
            rowComp += 5
    if bbox[1] < 0:
        if np.count_nonzero(threshold[w:y, z-10]) < 10:
            colComp -= 5
    
    rowComp = np.int32(rowComp)
    colComp = np.int32(colComp)
    
    return rowComp, colComp

In [28]:
def svm_loss_vectorized(W, X, y, reg):
  
    loss = 0.0
    dW = np.zeros(W.shape) # initialize the gradient as zero
    num_train = X.shape[0]

    scores = X.dot(W)
    yi_scores = scores[np.arange(scores.shape[0]), y]  # http://stackoverflow.com/a/23435843/459241
    margins = np.maximum(0, scores - np.reshape(yi_scores, (-1, 1)) + 1)
    margins[np.arange(num_train), y] = 0
    loss = np.mean(np.sum(margins, axis=1))
    loss += reg * np.sum(W * W)
    
    binary = margins
    binary[margins > 0] = 1
    row_sum = np.sum(binary, axis=1)
    binary[np.arange(num_train), y] = -row_sum.T
    dW = np.dot(X.T, binary)

    # Average
    dW /= num_train

    # Regularize
    dW += reg * W

    return loss, dW


In [29]:
def inViewCheck(bbox):
    place = "Inside"
    
    if bbox[0] <    0    - 2 * bbox_l/4:
        place = "up-Out"
    if bbox[0] > frame_l - 2 * bbox_l/4:
        place = "down-Out"
    if bbox[1] <    0    - 2 * bbox_w/4:
        place = "left-Out"
    if bbox[1] > frame_w - 2 * bbox_w/4:
        place = "right-Out"
    
    return place

In [30]:
def onlineSVM(frame, bbox, W):
    if (bbox[0] > 0 and bbox[0] < frame_l-bbox_l) and (bbox[1] > 0 and bbox[1] < frame_w-bbox_w): # This checks if child is fully inside or not
        if direction=="right": 
            if bbox[1] < frame_w-2*bbox_w:
                True_patch = frame1[bbox[0]:bbox[0]+bbox_l, bbox[1]:bbox[1]+bbox_w]
                Fals_patch = frame1[bbox[0]:bbox[0]+bbox_l, bbox[1]+bbox_w:bbox[1]+2*bbox_w]
            else:
                True_patch = frame1[bbox[0]:bbox[0]+bbox_l, bbox[1]:bbox[1]+bbox_w]
                Fals_patch = frame1[bbox[0]:bbox[0]+bbox_l, bbox[1]-bbox_w:bbox[1]]
        elif direction=="left":
            if bbox[1] > bbox_w:
                True_patch = frame1[bbox[0]:bbox[0]+bbox_l, bbox[1]:bbox[1]+bbox_w]
                Fals_patch = frame1[bbox[0]:bbox[0]+bbox_l, bbox[1]-bbox_w:bbox[1]]
            else:
                True_patch = frame1[bbox[0]:bbox[0]+bbox_l, bbox[1]:bbox[1]+bbox_w]
                Fals_patch = frame1[bbox[0]:bbox[0]+bbox_l, bbox[1]+bbox_w:bbox[1]+2*bbox_w]
        
        x_train = np.zeros([2, bbox_l*bbox_w*3])
        x_train[0, :] = np.reshape(True_patch, (1, -1))
        x_train[1, :] = np.reshape(Fals_patch, (1, -1))
        x_train = np.hstack([x_train, np.ones((x_train.shape[0], 1))])
        
        data_loss, data_dW = svm_loss_vectorized(W, x_train, [1, 0], 0.000005) # 0.0005
        W = W - 1e-7 * data_dW
        ret = True
    else:
        ret = False
    
    return ret, W

In [31]:
def bottomSearch(frame2, frame1_gray, frame2_gray, bbox):
    correctsVal = []
    correctsIdx = []
    for i in range(np.int16(frame_w/(bbox_w/2)) - 2): # for i in range(9):
        test_patch = frame2[frame_l - bbox_l:frame_l, i*np.int16(bbox_w/2):i*np.int16(bbox_w/2)+bbox_w, :]
        test_patch_1_gray = frame1_gray[frame_l - bbox_l:frame_l, i*np.int16(bbox_w/2):i*np.int16(bbox_w/2) + bbox_w]
        test_patch_2_gray = frame2_gray[frame_l - bbox_l:frame_l, i*np.int16(bbox_w/2):i*np.int16(bbox_w/2) + bbox_w]
        test = np.reshape(test_patch, (1, -1))
        test = np.hstack([test, np.ones((test.shape[0], 1))])
        
        subtFrames = cv.subtract(test_patch_1_gray, test_patch_2_gray)
        ret, threshold = cv.threshold(subtFrames, 25, 255, cv.THRESH_BINARY)
        
        if np.count_nonzero(threshold) > 1000:
            probability = np.max(test.dot(W))
            predict = np.argmax(test.dot(W))
            if predict == 1:
                correctsVal.append(probability)
                correctsIdx.append(i)
    if len(correctsIdx) != 0:
        bestCand = np.argmax(correctsVal)
        bbox = [frame_l - np.int16(bbox_l/2), correctsIdx[bestCand]*np.int16(bbox_w/2), bbox_l, bbox_w]
    
    return bbox

In [32]:
def leftSearch(frame2, frame1_gray, frame2_gray, bbox):
    correctsVal = []
    correctsIdx = []
    for i in range(np.int16(frame_l/(bbox_l/2)) - 2): # for i in range(9):
        test_patch = frame2[i*np.int16(bbox_l/2):i*np.int16(bbox_l/2)+bbox_l, 0:bbox_w, :]
        test_patch_1_gray = frame1_gray[i*np.int16(bbox_l/2):i*np.int16(bbox_l/2)+bbox_l, 0:bbox_w]
        test_patch_2_gray = frame2_gray[i*np.int16(bbox_l/2):i*np.int16(bbox_l/2)+bbox_l, 0:bbox_w]
        test = np.reshape(test_patch, (1, -1))
        test = np.hstack([test, np.ones((test.shape[0], 1))])
        
        subtFrames = cv.subtract(test_patch_1_gray, test_patch_2_gray)
        ret, threshold = cv.threshold(subtFrames, 25, 255, cv.THRESH_BINARY)
        
        if np.count_nonzero(threshold) > 1000:
            probability = np.max(test.dot(W))
            predict = np.argmax(test.dot(W))
            if predict == 1:
                correctsVal.append(probability)
                correctsIdx.append(i)
    if len(correctsIdx) != 0:
        bestCand = np.argmax(correctsVal)
        bbox = [0 - np.int16(bbox_w/2), correctsIdx[bestCand]*np.int16(bbox_l/2), bbox_l, bbox_w]
    
    return bbox

In [45]:
#thewriter = csv.writer(f)

# For ouput the video somewhere
fourcc = cv.VideoWriter_fourcc(*'XVID') # Video codec parameter
out = cv.VideoWriter('Output.avi', fourcc, 12.46, (1920, 1080))

# Reading Video
cap = cv.VideoCapture('mohamad sadra dabiri.mp4')

# First frame operations
ret, frame1= cap.read()

# Define an initial bounding box
#bbox = np.array([528, 586, 540, 320]) # amir mehdi rajabi
#bbox = np.array([100, 100, 580, 360]) # Dorsa Ebrahimi Catted Video
#bbox = np.array([750, 460, 640, 360]) # Ehsan Abedi
#bbox = np.array([0, 600, 600, 340]) # Salma Javanmard Ghadiri
#bbox = np.array([100, -100, 600, 340]) # Anita Kheiri
#bbox = np.array([160, 1600, 560, 340]) # Aria Mansouri 01
#bbox = np.array([80, 1450, 560, 340]) # Aria Mansouri 02
#bbox = np.array([480, 0, 580, 380]) # Yalda Jahangiri
#bbox = np.array([400, 1700, 800, 440]) # Yazdan Jahangiri
#bbox = np.array([500, 1100, 700, 460]) # Yasna Mirabi
#bbox = np.array([260, 400, 540, 400]) # Yasna Mirabi

# Uncomment the line below to select a different bounding box
bbox = cv.selectROI(frame1, False)
bbox = np.array([bbox[1], bbox[0], bbox[3], bbox[2],])
bbox_l, bbox_w = bbox[2], bbox[3]
position = np.array([bbox[0] + bbox_l/2, bbox[1] + bbox_w/2])

frame1_gray = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)

cv.destroyAllWindows()

In [46]:
ori_res, prv_ori = 10, 0
counter = 0
direction = "right"
frame_l, frame_w = np.size(frame1_gray, 0), np.size(frame1_gray, 1)
# generate a random SVM weight matrix of small numbers
W = np.random.randn(bbox_l*bbox_w*3 + 1, 2) * 0.0001 

In [47]:
# Playing the video
while(cap.isOpened()):
#for x in range(200):
    
    ret, frame2 = cap.read() # Reading next frame
    if ret == 0: # Check if next frame still exist or not (the video has been ended or not)
        break
    frame2_gray = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY) # Convert RGB image(frame) to grayscale
    
    # Check if target is going to right or left and save the state in variable named direction
    if prv_ori > 90 and prv_ori <= 270:
        direcion = "left"
    else:
        direcion = "right"
    
    inOrOut = inViewCheck(bbox) # This function is for checking if the child is in view or not
    
    if inOrOut == "Inside":
        
        ret, W = onlineSVM(frame1, bbox, W) # This function trains child patch when child is in vecinity of center of view
        
        u_crop, v_crop = childFlow_1(frame1_gray, frame2_gray, bbox, direction)
        domOri, magHist = DomOriMag(u_crop, v_crop, ori_res)
        magHist[0:np.uint8(magHist.shape[0]/10)] = 0
        domMag = np.argmax(magHist)
        
        # In this part, we want to save some of our previous direction to compare with our new one and if the onlie direction
        # completely differs from what we get in our previous loops, we will ignore that and consider our saved previous
        # orientation except that
        if domMag > 50: # You can remove this if you don't want to check magnitude condition if you want to consider orientation
                        # memory for all kind of movements
            domOri, counter = oriJumpReduct(domOri, prv_ori, counter) # To detect orientation false calculations
        else:
            counter = 0
    
        rowMove, colMove = MagOriToMove(domMag, domOri) # This function convert (Magnitude, Orintation) data to diaplacement of 
                                                        # our bounding box
    
        outImg = np.copy(frame1)
        cv.rectangle(outImg, (bbox[1], bbox[0]), (bbox[1] + bbox[3], bbox[0] + bbox[2]), (255, 15, 255), 10)
        out.write(outImg)
        cv.namedWindow('Out', cv.WINDOW_NORMAL)
        cv.imshow('Out', outImg)
        
        bbox += np.array([rowMove, colMove, 0, 0])
    
        ################################################
        rowComp, colComp = MoveComp_1(frame1_gray, frame2_gray, domMag)
        
        bbox += np.array([rowComp, colComp, 0, 0])
        ################################################
        
    elif inOrOut == "down-Out":
        
        bbox = bottomSearch(frame2, frame1_gray, frame2_gray, bbox)
        
        out.write(frame1)
        cv.namedWindow('Out', cv.WINDOW_NORMAL)
        cv.imshow('Out', frame1)
        
    elif inOrOut == "left-Out":
        
        bbox = bottomSearch(frame2, frame1_gray, frame2_gray, bbox)
        
        out.write(frame1)
        cv.namedWindow('Out', cv.WINDOW_NORMAL)
        cv.imshow('Out', frame1)
    
    frame1_gray = frame2_gray
    frame1 = frame2
    prv_ori = domOri
    
    # Exit if ESC pressed
    k = cv.waitKey(1) & 0xff
    if k == 27 : break

cap.release()
out.release()
cv.destroyAllWindows()

In [166]:
#outImg = np.copy(frame1)
#cv.rectangle(outImg, (bbox[1], bbox[0]), (bbox[1] + bbox[3], bbox[0] + bbox[2]), (255, 15, 255), 10)

cv.namedWindow('Out1', cv.WINDOW_NORMAL)
cv.imshow('Out1', np.uint8(b))
cv.namedWindow('Out2', cv.WINDOW_NORMAL)
cv.imshow('Out2', Fals_patch)
cv.waitKey()
cv.destroyAllWindows()

In [337]:
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

p0 = cv.goodFeaturesToTrack(frame2_gray, mask = None, **feature_params)

In [14]:
p0.shape

(38, 1, 2)

In [82]:
#u_crop, v_crop = childFlow(frame1_gray, frame2_gray, bbox, direction)
#domOri, magHist = DomOriMag(u_crop, v_crop, ori_res)
u_crop, v_crop = childFlow_new(frame1_gray, frame2_gray, bbox, direction)
domOri, magHist = DomOriMag(u_crop, v_crop, ori_res)

cv.namedWindow('Out1', cv.WINDOW_NORMAL)
cv.imshow('Out1', frame1_gray)
cv.namedWindow('Out2', cv.WINDOW_NORMAL)
cv.imshow('Out2', frame2_gray)
cv.waitKey()
cv.destroyAllWindows()

In [360]:
lk_params = dict( winSize  = (15,15),
                  maxLevel = 4,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 3, 0.03))
    
bbox_new = np.copy(bbox)
    
# Check right side movement of bbox
if direction == "right":
    bbox_new = np.array([bbox[0], bbox[1], bbox_l, 2*bbox_w])
    bbox_new = np.clip(bbox_new, [0, 0, 0, 0], [frame_l, frame_w, frame_l, frame_w])
elif direction == "left":
    bbox_new = np.array([bbox[0], bbox[1]-bbox_w, bbox_l, 2*bbox_w])
    bbox_new = np.clip(bbox_new, [0, 0, 0, 0], [frame_l, frame_w, frame_l, frame_w])
        
# Now, we will calculate optical flow and take our crop from previous step
    
subtFrames = cv.absdiff(frame2_gray, frame1_gray)
ret, thresh = cv.threshold(subtFrames, 25, 255, cv.THRESH_BINARY)

#thresh_crop = thresh[bbox_new[0]:bbox_new[0]+bbox_new[2], bbox_new[1]:bbox_new[1]+bbox_new[3]]
#frame1_crop = frame1_gray[bbox_new[0]:bbox_new[0]+bbox_new[2], bbox_new[1]:bbox_new[1]+bbox_new[3]]
#frame2_crop = frame2_gray[bbox_new[0]:bbox_new[0]+bbox_new[2], bbox_new[1]:bbox_new[1]+bbox_new[3]]
flow_u = np.zeros((np.size(frame1_gray, 0), np.size(frame1_gray, 1)))
flow_v = np.zeros((np.size(frame1_gray, 0), np.size(frame1_gray, 1)))
    
ptsRow, ptsCol = np.nonzero(thresh)
pts0 = np.transpose(np.array([ptsRow, ptsCol], dtype=np.float32))
    
pts1, st, err = cv.calcOpticalFlowPyrLK(frame1_gray, frame2_gray, pts0, None, **lk_params)
e, f = np.nonzero(st)
good_pts0 = pts0[e]
good_pts1 = pts1[e]
    
u_v = (good_pts1 - good_pts0)
flow_u[np.int16(good_pts0[:, 0]), np.int16(good_pts0[:, 1])] = u_v[:, 0]
flow_v[np.int16(good_pts0[:, 0]), np.int16(good_pts0[:, 1])] = u_v[:, 1]
    
u_crop = flow_u[bbox_new[0]:bbox_new[0]+bbox_new[2], bbox_new[1]:bbox_new[1]+bbox_new[3]]
v_crop = flow_v[bbox_new[0]:bbox_new[0]+bbox_new[2], bbox_new[1]:bbox_new[1]+bbox_new[3]]
    

(106184, 2)
(106184, 1)
[[    0.   194.]
 [    0.   195.]
 [    0.   205.]
 ..., 
 [ 1065.   923.]
 [ 1065.   925.]
 [ 1066.   927.]]
(array([    53,     54,     55, ..., 105008, 105009, 105102], dtype=int64), array([0, 0, 0, ..., 0, 0, 0], dtype=int64))
